## Classification Power Encoding

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
#%matplotlib notebook
# %matplotlib inline
# %load_ext autoreload
# %autoreload 2
from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *

from brainpipe.statistics import *
from os import path

### Check all files dimensions

In [ ]:
st = study('Olfacto')

files = st.search('_odor_bad_bipo_sel_phys.npz', folder='database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
for fi in files:
    loadname = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/', fi)
    mat = np.load(loadname)
    x = np.load(loadname)['x']
    print (fi, x.shape, mat.files)

### Select only electrodes with 4 or 5 patients by AAL rois

In [ ]:
st = study('Olfacto')
path_data = path.join(st.path,'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
to_keep = ['Cingulum Ant', 'Frontal Inf Orb', 'Frontal Mid', 'Frontal Mid Orb',
       'Frontal Sup Orb', 'Fusiform', 'Hippocampus', 'Insula', 'Temporal Inf',
       'Temporal Mid', 'Temporal Sup']
#to_keep = ['Frontal Inf Orb', 'Frontal Mid Orb', 'Frontal Sup Orb', 'Hippocampus',
#       'Insula', 'Temporal Inf', 'Temporal Mid']
subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
conds = ['bad', 'good']

for su in subjects:
    for cond in conds:
        mat = np.load(path_data+su+'_odor_'+cond+'_bipo_sel_phys.npz')
        idx = np.where([mat['aal'][i][:-4] in to_keep for i in range(len(mat['aal']))])
        kwargs = {}
        kwargs['labels'], kwargs['x'], kwargs['aal'] = mat['labels'][idx], mat['x'][idx], mat['aal'][idx]
        kwargs['xyz'], kwargs['channels'], kwargs['sf'] = mat['xyz'][idx], mat['channels'][idx], mat['sf']
        kwargs['BA']=mat['BA']
        np.savez(path_data+su+'_odor_'+cond+'_bipo_sel_aal_4.npz',**kwargs)
        print(su,mat['x'][idx].shape)
        del mat

### Compute Power for 3 windows in time :: MEAN
    No baseline correction OR baseline = mean power along the trial
    Win1 = 0-1.5 / Win2 = 1.5-3 / Win3 = 3-4.5 (sec)

In [ ]:
from brainpipe.feat.utils._feat import _manageWindow
st = study('Olfacto')
subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
conds = ['bad', 'good']
bsl, width, step = None, 358, 51
baseline = 'None'

for su in subjects:
    for cond in conds:
        # Define power settings :
        kwargs = {} # Define an empty dictionnary to save all power parameters
        kwargs['f'] = [[0.1,1.5],[2,4],[4, 8], [8, 13], [13, 30], [30, 60], [60,120]] # Frequency vector
        kwargs['split'] = [None,None,None,None,None,None,6]
        fname = ['VLFC','delta','theta', 'alpha', 'beta', 'gamma1', 'gamma2'] # Name of each frequency
        kwargs['width'], kwargs['step'] = width, step # take power in 358 samples (700ms) windows width every 51 samples (100ms)
        
        # Load file :
        fi = su+'_odor_'+cond+'_bipo_sel_aal_4.npz'
        loadname = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/', fi)
        mat = np.load(loadname)
        #print(mat.files)
        x, sf, aal_labels = mat['x'], mat['sf'], mat['aal']
        n_elec, n_pts, n_trials = x.shape
        print ('baseline:', bsl)
        print ('--> compute power on : ', fi, cond,'shape', x.shape)
        
        # Define and save power objects :
        powObj_x = power(sf, n_pts, baseline=bsl, norm=3, **kwargs,method='hilbert1')
        win_all, time = _manageWindow(2816, width=width, step=step)
        kwargs['time'] = np.array(time) / 512
        kwargs['fname'], kwargs['s_aal'], kwargs['s_BA'] = fname, mat['aal'], mat['BA']
        kwargs['labels'], kwargs['channels'], kwargs['xyz'] = mat['labels'], mat['channels'], mat['xyz']
        kwargs['xpow'],  kwargs['xpow_pval']= powObj_x.get(x,n_jobs=-1)
        print(kwargs['xpow'].shape)
        save_x = path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi_AAL/', fi.replace('.npz', '_'+baseline+'_pow.npz'))
        np.savez(save_x, **kwargs)
        del kwargs['xpow'],  kwargs['xpow_pval'], kwargs['fname']
        del kwargs, x, sf, n_elec, n_trials

### Add BA labels to power files

In [ ]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('Olfacto')
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']
conds=['bad','good']
path_data = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
path_pow = path.join(st.path,'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/')
path_labels = path.join(st.path,'Visbrain/By_subject/')
bsl = 'None'

for su in subjects:
    for cond in conds:
        # Load the data
        filename = su+'_odor_'+cond+'_bipo_sel_phys_'+bsl+'_pow.npz'
        mat = np.load(path_pow+filename)
        print(mat.files)
        kwargs = {}
        kwargs['fname'], kwargs['aal'],kwargs['xyz'] = mat['fname'], mat['phys'], mat['xyz']
        kwargs['labels'], kwargs['channels'] = mat['labels'], mat['channels']
        kwargs['xpow'],  kwargs['xpow_pval'] = mat['xpow'], mat['xpow_pval']
        # Load the BA labels
        mat2 = np.load(path_data+su+'_odor_'+cond+'_bipo_sel_phys.npz')
        kwargs['BA'] = mat2['BA']
        # Update the datafile
        np.savez(path_pow+filename, **kwargs)
        mat3 = np.load(path_pow+filename)
        print(mat3.files)
        print(mat3['xpow'].shape, mat3['xyz'].shape, mat3['aal'].shape, mat3['BA'].shape)
    

### Add BA and AAL labels to Power files

In [ ]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('Olfacto')
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ']
conds=['bad','good']
path_data = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
path_pow = path.join(st.path,'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/')
path_labels = path.join(st.path,'Visbrain/By_subject/')
bsl = 'None'

for su in subjects:
    for cond in conds:
        # Load the data
        filename = su+'_odor_'+cond+'_bipo_sel_'+bsl+'_pow.npz'
        mat = np.load(path_pow+filename)
        print(mat.files)
        kwargs = {}
        kwargs['fname'], kwargs['xyz'] = mat['fname'], mat['xyz']
        kwargs['labels'], kwargs['channels'] = mat['labels'], mat['channels']
        kwargs['xpow'],  kwargs['xpow_pval'] = mat['xpow'], mat['xpow_pval']
        # Load the BA labels
        mat2 = np.load(path_data+su+'_odor_'+cond+'_bipo_sel_phys.npz')
        kwargs['BA'], kwargs['aal'] = mat2['BA'], mat2['aal']
        # Update the datafile
        np.savez(path_pow+filename, **kwargs)
        mat3 = np.load(path_pow+filename)
        print(mat3.files)
        print(mat3['xpow'].shape, mat3['xyz'].shape, mat3['aal'].shape, mat3['BA'].shape)
    

## Compute all power for classif REST
Modification of Time window & steps to compute Power

Compute power relative to baseline for good and bad odors 

In [ ]:
st = study('Olfacto')
#subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
conds = ['bad', 'good']
phases = ['odor']
bsl = None
for su in subjects:
    for phase in phases:
        for cond in conds:
            # Define power settings :
            kwargs = {} # Define an empty dictionnary to save all power parameters
            kwargs['f'] = [[0.1,1.5],[2,4],[4, 8], [8, 13], [13, 30], [30, 60], [60,120]] # Frequency vector
            fname = ['VLFC','delta','theta', 'alpha', 'beta', 'gamma1', 'gamma2'] # Name of each frequency
            kwargs['width'], kwargs['step'] = 358, 51 # take power in 358 samples (700ms) windows width every 51 samples (100ms)

            #358, 25 700ms with 50ms of overlap, 410, 51 800ms with 100 ms overlap

            # Load file :
            fi = su+'_'+phase+'_'+cond+'_bipo_sel.npz'
            loadname = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/', fi)
            mat = np.load(loadname)
            x = mat['x']
            # Choice of baseline 
            baseline = bsl
            print ('baseline:', baseline)
            print ('--> compute power on : ', fi, phase, cond,'shape', x.shape)
            sf = 512
            n_elec, n_pts, n_trials = x.shape      

            # Define and save power objects :
            powObj_x = power(sf, n_pts, baseline=baseline, norm=None, **kwargs,method='hilbert1')
            kwargs['fname'] = fname
            kwargs['labels'], kwargs['channels'], kwargs['xyz'] = mat['label'], mat['channel'], mat['xyz']
            kwargs['xpow'],  kwargs['xpow_pval']= powObj_x.get(x,n_jobs=-1)
            save_x = path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/', fi.replace('.npz', '_'+str(bsl)+'_pow.npz'))
            np.savez(save_x, **kwargs)
            del kwargs['xpow'],  kwargs['xpow_pval'], kwargs['fname']
            del kwargs, x, sf, n_elec, n_trials

### Check all power files dimensions

In [3]:
st = study('Olfacto')
files = st.search('500_pow.npz', folder='feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/')
for fi in files:
    loadname = path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/', fi)
    mat = np.load(loadname)
    x = np.load(loadname)['xpow']
    # nfreq, nelec, nwin, ntrials
    print (fi, x.shape)
    print(mat.files)

-> Olfacto loaded


FileNotFoundError: [Errno 2] No such file or directory: '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/'

## Trick to have the time vector associated with Pow

In [ ]:
from brainpipe.feat.utils._feat import _manageWindow

win_all, time = _manageWindow(2816, width=256, step=128)
time = np.array(time) / 512
time[:-3]

## Plot all power by elec 
balanced and unbalanced conditions

In [ ]:
%matplotlib notebook
#NORMALIZATION BY THE MEAN
st = study('Olfacto')
fnames = np.load(path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/SEMC_odor_good_bipo_sel_phys_None_500_pow.npz'))['fname']
good_power = np.load(path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/SEMC_odor_good_bipo_sel_phys_None_500_pow.npz'))['xpow']
bad_power = np.load(path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/SEMC_odor_bad_bipo_sel_phys_None_500_pow.npz'))['xpow']
print(good_power.shape, bad_power.shape,fnames[2])
elec_good_power = good_power[6,63,:-6,:]
elec_bad_power = bad_power[6,63,:-6,:]
print(elec_good_power.shape, elec_bad_power.shape)
times = np.arange(elec_good_power.shape[0]/512*100)
plt.plot(time,np.mean(elec_good_power, axis=1), color='m')
plt.plot(time,np.mean(elec_bad_power, axis=1), color='b')
plt.xticks(time[:-5])
#plt.ylim(0)

In [ ]:
%matplotlib notebook
#NO NORMALIZATION
st = study('Olfacto')
fnames = np.load(path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/LEFC_odor_good_bipo_sel_phys2_None_pow.npz'))['fname']
good_power = np.load(path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/LEFC_odor_good_bipo_sel_phys2_None_pow.npz'))['xpow']
bad_power = np.load(path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/LEFC_odor_bad_bipo_sel_phys2_None_pow.npz'))['xpow']
print(good_power.shape, bad_power.shape,fnames[2])
elec_good_power = good_power[6,21,:,:]
elec_bad_power = bad_power[6,21,:,:]
print(elec_good_power.shape, elec_bad_power.shape)
plt.plot(np.mean(elec_good_power, axis=1), color='m')
plt.plot(np.mean(elec_bad_power, axis=1), color='b')

## Classification across time for all freq bands and subjects
### Analysis subject by subject

In [ ]:
# Importing files 
st = study('Olfacto')
pathfiles = path.join(st.path, 'feature/6_Power_E1E2_Odor_Good_Bad_700_100/')
elecfiles = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_concatOK/')
path2save = path.join(st.path, 'classified/7_Classif_Power_Good_Bad_across_time_700ms_step100ms_subset/')

#subjects = ['CHAF','SEMC', 'VACJ','PIRJ', 'MICP', 'LEFC']
#freq_bands = ['delta', 'theta', 'alpha', 'beta', 'gamma30-60', 'gamma60-120']

freq = 1
elec_num = 24
classif = 'lda'

#files & data to load
bad_data = np.load(path.join(pathfiles, 'PIRJ_concat_odor_bad_bipo_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec
good_data = np.load(path.join(pathfiles, 'PIRJ_concat_odor_good_bipo_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec
print ('bad shape: ', bad_data.shape, 'good shape: ', good_data.shape)
elec = np.load(path.join(elecfiles, 'PIRJ_concat_odor_bad_bipo.npz'))['channel'][elec_num]
elec_label = np.load(path.join(elecfiles, 'PIRJ_concat_odor_bad_bipo.npz'))['label'][elec_num]
freq_name = np.load(path.join(pathfiles, 'PIRJ_concat_odor_bad_bipo_power.npz'))['fname'][freq]
print ('elec ', elec, 'elec_label ', elec_label)

#create a data matrix, concatenate along the trial dimension
bad_good = np.concatenate((bad_data, good_data), axis=1)
bad_good = bad_good.swapaxes(0,1)
print ('Size of the concatenated data: ', bad_good.shape, 'Number of features : ', bad_good.shape[1])

#create label vector (0 for rest and 1 for odor)
label = [0]*bad_data.shape[1] + [1]*good_data.shape[1]
print ('Size of label for classif: ', len(label))

# Define a cross validation:
cv = defCv(label, n_folds=10, cvtype='skfold', rep=10)

# Define classifier technique
clf = defClf(label, clf=classif, n_knn=5,kern='rbf') #,n_tree=200, random_state=100)

#Classify rest and odor
cl = classify(label, clf=clf, cvtype=cv)

# Evaluate the classifier on data:
da,pvalue,daperm = cl.fit(bad_good, n_perm=100,method='label_rnd',mf=False)
print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape, 'daperm', daperm.shape)
#print (da)

# Plot Power analysis + stats and Decoding accuracy + max stat
## corrected across time for the da

In [1]:
from matplotlib.ticker import ScalarFormatter, MaxNLocator
%matplotlib notebook
#Import odor power corrected with a baseline
#odor_corr_data = np.load(path.join(pathfiles, 'PIRJ_E1E2_concat_all_bipo_odor_corr_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec

#Parameters figure
xfmt = ScalarFormatter(useMathText=True)
xfmt.set_powerlimits((0,3))
fig = plt.figure(1,figsize=(15,4))
step = 3500/ bad_data.shape[0]
time = np.arange(-500, 3000, step)
title = 'Power and DA for '+str(freq_name)+' PIRJ '+str(elec_label)+' ('+str(elec_num)+')'
fig.suptitle(title, fontsize=16)

# Plot the power and da :
plt.subplot(131)
plt.plot(time, bad_data, color='b', marker='*',)
plt.plot(time, good_data, color='r', marker='o',)
plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

#concatenate odor (red) then rest (blue)
plt.subplot(132)
BorderPlot(time, bad_good, y=label, xlabel='Time (ms)',ylabel = r' $\mu$V',
           alpha=0.3,linewidth=2,color=['r','b'],kind='sem')
rmaxis(plt.gca(), ['right', 'top'])
addLines(plt.gca(),hColor= ['dimgrey'], hLines=[0], hWidth=[1], hShape=['-'],vLines=[1],vWidth=[1.5],vShape=['-'])
plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

plt.subplot(133)
title = str(freq_name)+' da : PIRJ '+str(elec_label)+' ('+str(elec_num)+')'
BorderPlot(time, da, color='darkslateblue', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
           ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
rmaxis(plt.gca(), ['right', 'top'])
addLines(plt.gca(), vLines=[1],vWidth=[1.5],vShape=['-'], vColor=['dimgrey'],
        hLines=[50], hColor=['#000000'], hWidth=[2])
plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
plt.plot(time, daperm.max()*np.ones((len(time),1)), '--', color='dimgrey', linewidth=2, label='p < .01')
print(daperm.max())
plt.legend(loc=0, handletextpad=0.1, frameon=False)
plt.gca().yaxis.set_major_formatter(xfmt)


ImportError: /home/karim/anaconda3/lib/python3.5/lib-dynload/../../libz.so.1: version `ZLIB_1.2.9' not found (required by /home/karim/anaconda3/lib/python3.5/site-packages/matplotlib/../../.././libpng16.so.16)

In [ ]:
#Generate the figure and save the plot
title = 'Classif_Encoding_'+cond+'_'+su+'_'+ freq+'_p<0.001'
fig1 = plt.figure(1, figsize=(20,10))
fig1.suptitle(title, fontsize="x-large")
cl.daplot(da_mean, daperm=daperm, chance_method='perm', rmax=['top', 'right'],
dpax=['bottom', 'left'], cmap='magma', ylim=[40,100], chance_unique=True, chance_level = 0.001,
chance_color='darkgreen',)

fname = path.join(path2save, su + '_'+cond+'_'+freq+'_0.001.png')
fig1.savefig(fname, dpi=300, bbox_inches='tight')
print ('saving --»' ,fname)
cl.info.to_excel(path2save+su+'_'+cond+'_'+freq+'_'+classifier+'.xlsx')
plt.clf()
plt.close()
del da_mean, elec

## Check da perm and da

In [ ]:
st = study('Olfacto')
files = st.search('da_Odor', folder='classified/5_Classif_Windows_700ms_Encoding_across_time__SVM_linear/')
for fi in files:
    loadname = path.join(st.path, 'classified/5_Classif_Windows_700ms_Encoding_across_time__SVM_linear/', fi)
    x = np.load(loadname)
    print (fi, x.shape, x[5])